# RAVE
**RAVE** is a variational autoencoder for fast and high-quality neural audio synthesis by Antoine Caillon and Philippe Esling.
\
\
[Article on arxiv](https://arxiv.org/abs/2111.05011) & [Source code on Github](https://github.com/acids-ircam/RAVE)
\
---

<small>*This notebook is based on the one from [Hexorcismos](https://colab.research.google.com/drive/1ih-gv1iHEZNuGhHPvCHrleLNXvooQMvI?usp=sharing) and comes with a few additional hints and comments.* 
\
*Last updated 10.03.2023.*

# Setup

In [ ]:
#@title **Colab setup** (run this each time you want to start or resume training)
#@markdown **Select the version of RAVE you want to use.** \ 
#@markdown *It's recommended to pin this notebook to a version/ release tag of RAVE, to avoid errors when resuming training.* https://github.com/acids-ircam/RAVE/tags
version = "2.1.11" #@param {type:"string"}

#Install Conda
!mkdir /content/download
%cd /content/download
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /content/miniconda

#Install RAVE
!/content/miniconda/bin/pip install acids-rave==$version

#Upgrade ipython ipykernel, install ffmpeg
!/content/miniconda/bin/pip install --upgrade ipython ipykernel
!/content/miniconda/bin/conda install ffmpeg --yes

In [ ]:
#@title **Mount Google Drive**

from google.colab import drive
drive.mount('/content/drive')

# Preparation

In [ ]:
#@title **Define variables** (run this each time or when you change something)

#@markdown **1. Add the path to your working folder**\
#@markdown *This should be the main folder for all the generated data from pre processing and training. Make sure this is on your Google Drive, which is mounted to /content/drive/MyDrive* 
yourfolder = "/content/drive/MyDrive/myfolder/" #@param {type:"string"}
#@markdown ---
#@markdown **2. Add the path to your training dataset** *- aka the audio files you want RAVE to train on*
dataset = "/content/drive/MyDrive/myfolder/dataset/" #@param {type:"string"}
#@markdown ---
#@markdown **3. Add the path to where the data from pre processing should be stored**\ 
#@markdown *This should create a lock.mdb, data.mdb and metadata.yaml file in your folder.*
processed = "/content/drive/MyDrive/myfolder/meta/" #@param {type:"string"}
#@markdown ---
#@markdown **4. Give your training a name**\
#@markdown *Avoid blanks/ spaces. A subfolder is created in your main folder with your training's name and a string suffix. Your model data and checkpoints will be stored inside this folder.*
name = "training" #@param {type:"string"}
#@markdown ---
#@markdown **5. Select configuration parameters.** *Hint: there are various other configurations available. Use 'rave train --helpfull' to get a complete overview.*
architecture = "v2.gin" #@param ["v2.gin", "v1.gin", "discrete.gin", "onnx.gin", "raspberry.gin"]
regularization = "wasserstein" #@param ["default", "wasserstein", "spherical"]

In [ ]:
#@title **Pre process dataset** (only needed once per dataset)

!mkdir $processed

#Optional: you can resample your material to a fixed sample rate. Files will be store in a out_<selected-sample-rate> subfolder. Make sure to change your dataset folder in the section above and rerun it. 
#samplerate = "44100" #@param ["22050", "44100", "48000"]
#resample --sr $samplerate --augment

!/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $processed

#Use '--lazy' on a heterogenous file set e.g. with different formats, audio settings etc. - see also https://github.com/acids-ircam/RAVE#dataset-preparation
#!/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $processed --lazy 

In [ ]:
#@title **Load Tensorboard for monitoring training** (each training)
#@markdown **Tensorboard will be loaded from inside your working folder.**\
#@markdown *Works best with Chrome. Disable ad and script blockers. Some monitoring data is only shown in the second phase of the training.* 

%cd $yourfolder
%load_ext tensorboard
%tensorboard --logdir . --bind_all

# Training

In [ ]:
#@title **Initial training** (start with this)
%cd $yourfolder

#Initiate training
!/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $processed --name $name

#In earlier versions of RAVE, when using Wasserstein regularization, the latent space has 128 dimensions per default. Use "--override LATENT_SIZE=16" as seen below to change the dimensions to a different value. 
#!/content/miniconda/bin/rave train --config $architecture --config $regularization --override LATENT_SIZE=16 --db_path $processed --name $name

In [ ]:
#@title **Resume training** (continue with this)
#@markdown **Add the path to the most recent checkpoint folder in order to resume training.**\ 
#@markdown *These can be found in the /runs subfolder. Training name and parameters are taken from above section's settings. Make sure to run that section before resuming.*
checkpoint = "/content/drive/MyDrive/myfolder/runs/<yourtrainingnamewitharandomstringintheend>/version_<XX>/" #@param {type:"string"}

%cd $yourfolder

#Resume training
!/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $processed --name $name --ckpt $checkpoint

#In earlier versions of RAVE, when using Wasserstein regularization, the latent space has 128 dimensions per default. Use "--override LATENT_SIZE=16" as seen below to change the dimensions to a different value. 
#!/content/miniconda/bin/rave train --config $architecture --config $regularization --override LATENT_SIZE=16 --db_path $processed --name $name --ckpt 

# Export

In [ ]:
#@title **Export model**
#@markdown **Add the path to your model data.**\ 
#@markdown *This is a subfolder in /runs with your training name and a string suffix.* 
modelfolder = "/content/drive/MyDrive/myfolder/runs/<yourtrainingnamewitharandomstringintheend>/" #@param {type:"string"}

!/content/miniconda/bin/rave export --run $modelfolder --streaming